In [2]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import scipy.stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

first_days_of_month = [0,31,59,90,120,151,181,212,243,273,304,334]
months_str = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
months_str = months_str[9:12] + months_str[0:9]


In [175]:
def plot_sine(T0, Ta, phi, std_winter, std_summer):
    
    omega = 2*np.pi / 365
    t = np.linspace(0,365,365)
    T_median = T0 - Ta * np.sin(omega*t - phi)
    
    T0_1sigma = T0 + 0.25*(std_winter + std_summer)
    Ta_1sigma = Ta + 0.25*(std_summer - std_winter)
    T_1sigma = T0_1sigma - Ta_1sigma * np.sin(omega*t - phi)
    
    T0_2sigma = T0 + 0.5*(std_winter + std_summer)
    Ta_2sigma = Ta + 0.5*(std_summer - std_winter)
    T_2sigma = T0_2sigma - Ta_2sigma * np.sin(omega*t - phi)
    
    delta = np.zeros(3)
    ddelta_dT0 = np.zeros(3)
    percentiles = np.array([50,84,97.5])
    Tas = [Ta,Ta_1sigma,Ta_2sigma]
    T0s = [T0,T0_1sigma,T0_2sigma]
    tp = np.zeros(3)
    tm = np.zeros(3)
    
    for kk_p, p in enumerate(percentiles):
        
        T0_kk = T0s[kk_p]
        Ta_kk = Tas[kk_p]
        
        tp[kk_p] = (1/omega) * (np.pi + phi - np.arcsin(T0_kk / Ta_kk))
        tm[kk_p] = (1/omega) * (phi + np.arcsin(T0_kk / Ta_kk))
        
        if tp[kk_p] > tm[kk_p]:
            delta[kk_p] = tp[kk_p] - tm[kk_p]
            ddelta_dT0[kk_p] = -2*(1/omega) * (Ta_kk**2 - T0_kk**2)**(-0.5)
        else:
            delta[kk_p] = 0
            ddelta_dT0[kk_p] = np.nan
            
    zs = np.arange(0,2.1,0.2)
    ddelta_dT0_2 = np.zeros(len(zs))
    for kk_z, z in enumerate(zs): #diff z-scores

        T0_this = T0 + z*0.25*(std_winter + std_summer)
        Ta_this = Ta + z*0.25*(std_summer - std_winter)
        T_this = T0_this - Ta_this * np.sin(omega*t - phi)
        
        tp_this = (1/omega) * (np.pi + phi - np.arcsin(T0_this / Ta_this))
        tm_this = (1/omega) * (phi + np.arcsin(T0_this / Ta_this))
        
        if tp_this > tm_this:
            delta_this = tp_this - tm_this
            ddelta_dT0_this = -2*(1/omega) * (Ta_this**2 - T0_this**2)**(-0.5)
        else:
            delta_this = 0
            ddelta_dT0_this = np.nan
            
        ddelta_dT0_2[kk_z] = ddelta_dT0_this
            
    ##### visualize
    nrows = 1
    ncols = 2
    
    fig, axes = plt.subplots(nrows = nrows, ncols = ncols, figsize = (6*ncols, 4*nrows))
    
    ##### temp
    ax = axes[0]
    
    #2sigma
    y1 = T_2sigma
    y2 = T_median-(T_2sigma-T_median)
    ax.fill_between(x = t, y1 = y1, y2 = y2, alpha = 0.2, label = '2$\sigma$')
    
    #1sigma
    y1 = T_1sigma
    y2 = T_median-(T_1sigma-T_median)
    ax.fill_between(x = t, y1 = y1, y2 = y2, alpha = 0.5, label = '1$\sigma$')

    #median
    ax.plot(t,T_median, 'k', label = 'Median')  
    
    ax.plot([tm[0],tm[0]], [0,15], 'k--')
    ax.plot([tp[0],tp[0]], [0,15], 'k--')
    ax.annotate("", xy=(tm[0], 15), xytext=(tp[0], 15),arrowprops=dict(arrowstyle="<->"))
    ax.text(x = (tm[0]+tp[0]) / 2, y = 17, s = '$\Delta_{50}$')
    ax.plot([0,364],[0,0], 'k', linewidth = 0.5)

    ax.set_title('Temperature [$\degree C$]')
    ax.set_ylabel('Temperature [$\degree C$]')
    ax.set_xlim(left = 0, right = 365)
    ax.set_ylim(bottom = -25, top = 20)
    ax.grid(True)
    ax.set_axisbelow(True)
    ax.set_xticks(first_days_of_month)
    ax.set_xticklabels(months_str, rotation = 0)
    ax.legend()
    
    ##### sensitivity
    ax = axes[1]
#     ax.plot(percentiles,ddelta_dT0, marker = 'o')
    p_values = (1 - scipy.stats.norm.sf(abs(zs)))*100#scipy.stats.norm.sf(abs(zs))*2
    ax.plot(p_values,ddelta_dT0_2, marker = 'o')
    
    ax.set_ylim(top = -0, bottom = -30)
    ax.set_xlim(left = 47, right = 100)
    ax.set_xlabel('k: Temperature percentile')
    ax.set_ylabel('$\partial \Delta_k / \partial T_0$')
    ax.set_title('$\partial \Delta_k / \partial T_0$')
    
#     plt.savefig('test_widget.png', dpi = 300, bbox_inches = 'tight')
    
    plt.show()
    


In [177]:
slider = interact(plot_sine,
                  T0 = widgets.FloatSlider(value = 0, min = -10, max = 10),
                  Ta = widgets.FloatSlider(value = 10, min = 5, max = 20),
                  phi = widgets.FloatSlider(value = 0.3, min = -1, max = 1),
                  std_winter = widgets.FloatSlider(value = 7, min = 1, max = 10),
                  std_summer = widgets.FloatSlider(value = 3, min = 1, max = 10)
                 )



interactive(children=(FloatSlider(value=0.0, description='T0', max=10.0, min=-10.0), FloatSlider(value=10.0, d…